In [1]:
# O
import openai
import os


import numpy as np

# R
import re

# pdfplumber
import pdfplumber

import json
import pandas as pd

from transformers import BertTokenizer, BertForQuestionAnswering
from elasticsearch import Elasticsearch, helpers

import pickle


In [2]:
import sklearn
print(sklearn.__version__)

1.4.2


In [3]:
# Load CSV data
file_path_advanced = "data/advanced_team_stats_2024.csv"
file_path_clutch = "data/clutch_team_stats_2024.csv"
file_path_player = "data/nba_2024_player_data.csv"
data_advanced = pd.read_csv(file_path_advanced)
data_clutch = pd.read_csv(file_path_clutch)
data_player = pd.read_csv(file_path_player)
# There are cases where players have shot 0 field goals resulting in null values in the data table - fixing this by replacing them with 0
data_player = data_player.fillna(0)

#### Dicts

In [4]:
advanced_abbreviation_dict = {
    "W": ["Wins", "Number of games won"],
    "L": ["Losses", "Number of games lost"],
    "PW": ["Pythagorean Wins", "Expected wins based on points scored and points allowed"],
    "PL": ["Pythagorean Losses", "Expected losses based on points scored and points allowed"],
    "MOV": ["Margin of Victory", "Average point differential per game"],
    "SOS": ["Strength of Schedule", "Measure of the strength of opponents"],
    "SRS": ["Simple Rating System", "Combines MOV and SOS"],
    "ORtg": ["Offensive Rating", "Points scored per 100 possessions"],
    "DRtg": ["Defensive Rating", "Points allowed per 100 possessions"],
    "NRtg": ["Net Rating", "Difference between ORtg and DRtg"],
    "Pace": ["Pace", "Number of possessions per 48 minutes"],
    "FTr": ["Free Throw Rate", "Free throws attempted per field goal attempt"],
    "3PAr": ["Three-Point Attempt Rate", "Percentage of field goal attempts from three-point range"],
    "TSpct": ["True Shooting Percentage", "Efficiency of all shooting attempts"],
    "Off_eFGpct": ["Effective Field Goal Percentage for Offense", "Adjusted field goal percentage accounting for three-pointers"],
    "Off_TOVpct": ["Turnover Percentage for Offense", "Turnovers per 100 possessions"],
    "Off_ORBpct": ["Offensive Rebound Percentage", "Percentage of available offensive rebounds grabbed"],
    "Off_FT/FGA": ["Free Throws Made per Field Goal Attempt", "Ratio of free throws made to field goal attempts"],
    "Def_eFGpct": ["Effective Field Goal Percentage for Defense", "Opponent's adjusted field goal percentage"],
    "Def_TOVpct": ["Turnover Percentage for Defense", "Opponent's turnovers per 100 possessions"],
    "Def_DRBpct": ["Defensive Rebound Percentage", "Percentage of available defensive rebounds grabbed"],
    "Def_FT/FGA": ["Free Throws Made per Field Goal Attempt for Defense", "Opponent's ratio of free throws made to field goal attempts"],
    "Arena": ["Home of the", "Home"],
    "Attend": ["Attendance, Overall Attendance"],
    "Attend/G": ["Attendance per Game, Average Attendance"],
}

In [5]:
clutch_abbreviation_dict = {
    "TEAM": ["Team", "Name of the team"],
    "GP": ["Games Played", "Number of games played"],
    "W": ["Wins", "Number of games won"],
    "L": ["Losses", "Number of games lost"],
    "WINpct": ["Win Percentage", "Winning percentage"],
    "MIN": ["Minutes", "Average minutes played"],
    "PTS": ["Points", "Average points scored"],
    "FGM": ["Field Goals Made", "Average field goals made"],
    "FGA": ["Field Goals Attempted", "Average field goals attempted"],
    "FGpct": ["Field Goal Percentage", "Field goal percentage"],
    "3PM": ["Three-Point Field Goals Made", "Average three-point field goals made"],
    "3PA": ["Three-Point Field Goals Attempted", "Average three-point field goals attempted"],
    "3Ppct": ["Three-Point Field Goal Percentage", "Three-point field goal percentage"],
    "FTM": ["Free Throws Made", "Average free throws made"],
    "FTA": ["Free Throws Attempted", "Average free throws attempted"],
    "FTpct": ["Free Throw Percentage", "Free throw percentage"],
    "OREB": ["Offensive Rebounds", "Average offensive rebounds"],
    "DREB": ["Defensive Rebounds", "Average defensive rebounds"],
    "REB": ["Rebounds", "Average total rebounds"],
    "AST": ["Assists", "Average assists"],
    "TOV": ["Turnovers", "Average turnovers"],
    "STL": ["Steals", "Average steals"],
    "BLK": ["Blocks", "Average blocks"],
    "BLKA": ["Blocks Against", "Average blocks against"],
    "PF": ["Personal Fouls", "Average personal fouls"],
    "PFD": ["Personal Fouls Drawn", "Average personal fouls drawn"],
    "+/-": ["Plus/Minus", "Plus/minus rating"]
}

In [6]:
playoff_player_abbreviation_dict = {
    "Rk": ["Rank", "Player rank"],
    "Player": ["Player", "Name of the player"],
    "Age": ["Age", "Player's age"],
    "G": ["Games Played", "Number of games played"],
    "MP": ["Minutes Played", "Total minutes played"],
    "PER": ["Player Efficiency Rating", "Player efficiency rating"],
    "TSpct": ["True Shooting Percentage", "True shooting percentage"],
    "3PAr": ["Three-Point Attempt Rate", "Three-point attempt rate"],
    "FTr": ["Free Throw Rate", "Free throw rate"],
    "ORBpct": ["Offensive Rebound Percentage", "Offensive rebound percentage"],
    "DRBpct": ["Defensive Rebound Percentage", "Defensive rebound percentage"],
    "TRBpct": ["Total Rebound Percentage", "Total rebound percentage"],
    "ASTpct": ["Assist Percentage", "Assist percentage"],
    "STLpct": ["Steal Percentage", "Steal percentage"],
    "BLKpct": ["Block Percentage", "Block percentage"],
    "TOVpct": ["Turnover Percentage", "Turnover percentage"],
    "USGpct": ["Usage Percentage", "Usage percentage"],
    "OWS": ["Offensive Win Shares", "Offensive win shares"],
    "DWS": ["Defensive Win Shares", "Defensive win shares"],
    "WS": ["Win Shares", "Total win shares"],
    "WS/48": ["Win Shares per 48 Minutes", "Win shares per 48 minutes"],
    "OBPM": ["Offensive Box Plus/Minus", "Offensive box plus/minus", "Offensive Box Plus-Minus"],
    "DBPM": ["Defensive Box Plus/Minus", "Defensive box plus/minus", "Defensive Box Plus-Minus"],
    "BPM": ["Box Plus/Minus", "Box plus/minus", "Box Plus-Minus"],
    "VORP": ["Value Over Replacement Player", "Value over replacement player"],
    "Team": ["Team", "Name of the team"]
}

#### Conference Data for possible data Training

In [7]:
def extract_teams_from_pdf(pdf_path):
    teams_data = {
        "Eastern Conference": {
            "Atlantic Division": [],
            "Central Division": [],
            "Southeast Division": []
        },
        "Western Conference": {
            "Northwest Division": [],
            "Pacific Division": [],
            "Southwest Division": []
        }
    }

    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()

        current_conference = None
        current_division = None

        for line in text.split('\n'):
            line = line.strip()
            if "Conference" in line:
                current_conference = line
            elif "Division" in line:
                current_division = line.replace(":", "")  # remove the colon
            elif line.startswith('o'):
                team_name = line[2:]
                if current_conference and current_division:
                    teams_data[current_conference.replace(":", "")][current_division].append(team_name)

    return teams_data

pdf_path = 'data/Conference_Teams.pdf'
teams_data = extract_teams_from_pdf(pdf_path)
print(teams_data)

{'Eastern Conference': {'Atlantic Division': ['Boston Celtics', 'Brooklyn Nets', 'New York Knicks', 'Philadelphia 76ers', 'Toronto Raptors'], 'Central Division': ['Chicago Bulls', 'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Milwaukee Bucks'], 'Southeast Division': ['Atlanta Hawks', 'Charlotte Hornets', 'Miami Heat', 'Orlando Magic', 'Washington Wizards']}, 'Western Conference': {'Northwest Division': ['Denver Nuggets', 'Minnesota Timberwolves', 'Oklahoma City Thunder', 'Portland Trail Blazers', 'Utah Jazz'], 'Pacific Division': ['Golden State Warriors', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Phoenix Suns', 'Sacramento Kings'], 'Southwest Division': ['Dallas Mavericks', 'Houston Rockets', 'Memphis Grizzlies', 'New Orleans Pelicans', 'San Antonio Spurs']}}


In [8]:
def extract_conference_data(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()

    return text

pdf_path = 'data/Conference_Teams.pdf'
conference_text = extract_conference_data(pdf_path)

In [9]:
def parse_conference_data(text):
    conference_data = {}
    current_conference = ""
    current_division = ""

    for line in text.split('\n'):
        line = line.strip()
        if "Conference" in line:
            current_conference = line
            conference_data[current_conference] = {}
        elif "Division" in line:
            current_division = line.split(':')[0]
            conference_data[current_conference][current_division] = []
        elif line.startswith('o'):
            team = line[2:].strip()
            conference_data[current_conference][current_division].append(team)

    return conference_data

conference_data = parse_conference_data(conference_text)

#### Elasticsearch

In [10]:
# Initialize Elasticsearch client
try:
    es = Elasticsearch(
        hosts=[{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
        http_auth=('elastic', 'changeme'),
        verify_certs=False
    )
    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Could not connect to Elasticsearch")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

Connected to Elasticsearch


C:\Users\Ray\AppData\Local\Temp\ipykernel_21960\2701087776.py:3: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(
C:\Users\Ray\AppData\Local\Temp\ipykernel_21960\2701087776.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if es.ping():


In [11]:
import warnings
warnings.filterwarnings('ignore')

#### Mapping

In [12]:
advanced_mappings = {
    "mappings": {
        "properties": {
            "Team": {"type": "text"},
            "W": {"type": "integer"},
            "L": {"type": "integer"},
            "PW": {"type": "integer"},
            "PL": {"type": "integer"},
            "MOV": {"type": "float"},
            "SOS": {"type": "float"},
            "SRS": {"type": "float"},
            "ORtg": {"type": "float"},
            "DRtg": {"type": "float"},
            "NRtg": {"type": "float"},
            "Pace": {"type": "float"},
            "FTr": {"type": "float"},
            "3PAr": {"type": "float"},
            "TSpct": {"type": "float"},
            "Off_eFGpct": {"type": "float"},
            "Off_TOVpct": {"type": "float"},
            "Off_ORBpct": {"type": "float"},
            "Off_FT/FGA": {"type": "float"},
            "Def_eFGpct": {"type": "float"},
            "Def_TOVpct": {"type": "float"},
            "Def_DRBpct": {"type": "float"},
            "Def_FT/FGA": {"type": "float"},
            "Arena": {"type": "text"},
            "Attend": {"type": "integer"},
            "Attend/G": {"type": "integer"},
        }
    }
}

In [13]:
clutch_mappings = {
    "mappings": {
        "properties": {
            "TEAM": {"type": "text"},
            "GP": {"type": "integer"},
            "W": {"type": "integer"},
            "L": {"type": "integer"},
            "WINpct": {"type": "float"},
            "MIN": {"type": "float"},
            "PTS": {"type": "float"},
            "FGM": {"type": "float"},
            "FGA": {"type": "float"},
            "FGpct": {"type": "float"},
            "3PM": {"type": "float"},
            "3PA": {"type": "float"},
            "3Ppct": {"type": "float"},
            "FTM": {"type": "float"},
            "FTA": {"type": "float"},
            "FTpct": {"type": "float"},
            "OREB": {"type": "float"},
            "DREB": {"type": "float"},
            "REB": {"type": "float"},
            "AST": {"type": "float"},
            "TOV": {"type": "float"},
            "STL": {"type": "float"},
            "BLK": {"type": "float"},
            "BLKA": {"type": "float"},
            "PF": {"type": "float"},
            "PFD": {"type": "float"},
            "+/-": {"type": "float"},
        }
    }
}


In [14]:
playoff_player_mappings = {
    "mappings": {
        "properties": {
            "Rk": {"type": "integer"},
            "Player": {"type": "text"},
            "Age": {"type": "integer"},
            "G": {"type": "integer"},
            "MP": {"type": "integer"},
            "PER": {"type": "float"},
            "TSpct": {"type": "float"},
            "3PAr": {"type": "float"},
            "FTr": {"type": "float"},
            "ORBpct": {"type": "float"},
            "DRBpct": {"type": "float"},
            "TRBpct": {"type": "float"},
            "ASTpct": {"type": "float"},
            "STLpct": {"type": "float"},
            "BLKpct": {"type": "float"},
            "TOVpct": {"type": "float"},
            "USGpct": {"type": "float"},
            "OWS": {"type": "float"},
            "DWS": {"type": "float"},
            "WS": {"type": "float"},
            "WS/48": {"type": "float"},
            "OBPM": {"type": "float"},
            "DBPM": {"type": "float"},
            "BPM": {"type": "float"},
            "VORP": {"type": "float"},
            "Team": {"type": "text"},
        }
    }
}

#### Indexing

In [15]:
def index_data(data, index_name, mappings):
    es.indices.create(index=index_name, ignore=400)
    actions = [
        {
            "_index": index_name,
            "_id": i,
            "_source": data.iloc[i].to_dict(),
        }
        for i in range(len(data))
    ]
    helpers.bulk(es, actions)

index_data(data_advanced, "advanced_stats", advanced_mappings)
index_data(data_clutch, "clutch_stats", clutch_mappings)
index_data(data_player, "player_stats", playoff_player_mappings)

In [16]:
# Verifying document count for each index
index_stats_advanced = es.indices.stats(index="advanced_stats")
print("Advanced Stats count after reindexing advanced_stats:", index_stats_advanced['_all']['primaries']['docs']['count'])

index_stats_clutch = es.indices.stats(index="clutch_stats")
print("Clutch Stats count after reindexing clutch_stats:", index_stats_clutch['_all']['primaries']['docs']['count'])

index_stats_player = es.indices.stats(index="player_stats")
print("Playoff Team Roster count after reindexing player_stats:", index_stats_player['_all']['primaries']['docs']['count'])


Advanced Stats count after reindexing advanced_stats: 30
Clutch Stats count after reindexing clutch_stats: 30
Playoff Team Roster count after reindexing player_stats: 337


In [17]:
index_name_team = "advanced_stats"
index_name_clutch = "clutch_stats"
index_name_player = "player_stats"

#### Search ES Method

In [18]:
# Function to search Elasticsearch
def search_es(query, index_name, field_name):
    body = {
        "query": {
            "bool": {
                "must": [
                    {"match": {field_name: query["team"]}} if "team" in query else {"match": {"Player": query["player"]}}
                ]
            }
        },
        "_source": query["stats"]
    }
    res = es.search(index=index_name, query=body['query'], _source=body['_source'])
    return res['hits']['hits']

#### Retrieval of statistical Data

In [19]:
# Function to process queries
def process_query(query):
    query = query.lower()

    # Extract team or player names
    team_names = []
    player_names = []

    for team in data_advanced['Team']:
        if team.lower() in query:
            team_names.append(team)

    for player in data_player['Player']:
        if player.lower() in query:
            player_names.append(player)

    if not team_names and not player_names:
        return "Teams or players not found in the dataset."

    # Determine if clutch stats are requested
    is_clutch = "clutch" in query

    # Use pattern matching to extract statistics
    stats = extract_statistics_with_pattern_matching(query)

    if "won and lost" in query:
        if "W" not in stats:
            stats.append("W")
        if "L" not in stats:
            stats.append("L")

    if not stats:
        return "No statistics found in the query."

    results = []
    if team_names:
        for team in team_names:
            es_query = {"team": team, "stats": stats}

            # Search in appropriate indices based on whether clutch stats are requested
            if is_clutch:
                search_results_clutch = search_es(es_query, index_name_clutch, "TEAM")
                combined_results = search_results_clutch
            else:
                search_results_advanced = search_es(es_query, index_name_team, "Team")
                search_results_clutch = search_es(es_query, index_name_clutch, "TEAM")
                combined_results = search_results_advanced + search_results_clutch

            if combined_results:
                result = pd.DataFrame([hit["_source"] for hit in combined_results])
                # Filter the DataFrame to only include the requested statistics
                result = result[[col for col in stats if col in result.columns]]
                results.append((team, result))

    if player_names:
        for player in player_names:
            es_query = {"player": player, "stats": stats}

            # Search in player index
            search_results_player = search_es(es_query, index_name_player, "Player")

            if search_results_player:
                result = pd.DataFrame([hit["_source"] for hit in search_results_player])
                # Filter the DataFrame to only include the requested statistics
                result = result[[col for col in stats if col in result.columns]]
                results.append((player, result))

    if not results:
        return "No matching data found in Elasticsearch."

    # Format the results as a sentence
    sentences = []
    for name, result in results:
        stats_str = ', '.join([f"{col}: {result[col].values[0]}" for col in result.columns if not pd.isna(result[col].values[0])])
        sentences.append(f"{name}: {stats_str}")

    return ' '.join(sentences)

In [20]:
def extract_statistics_with_pattern_matching(query):
    combined_abbreviation_dict = {**advanced_abbreviation_dict, **clutch_abbreviation_dict, **playoff_player_abbreviation_dict}
    extracted_stats = set()

    for key, values in combined_abbreviation_dict.items():
        for value in values + [key]:
            if re.search(rf'\b{re.escape(value.lower())}\b', query):
                extracted_stats.add(key)
                break

    return list(extracted_stats)

In [21]:
# Example queries
queries = [
    "Give me the Wins and ORtg for Boston Celtics",
    "What are the MOV and Defensive Rating of the Denver Nuggets?",
    "Show me the Pythagorean Wins and True Shooting Percentage for Oklahoma City Thunder",
    "How many games have the New York Knicks won and lost?",
    "Give me the Home and Offensive Rating for Boston Celtics and Denver Nuggets",
    "Give me the Clutch Win Percentage for Boston Celtics and Denver Nuggets",
    "Give me the Age of Jimmy Butler"
]

# Process each query and print the result
for query in queries:
    result = process_query(query)
    print(f"Query: {query}")
    print(result, "\n")

Query: Give me the Wins and ORtg for Boston Celtics
Boston Celtics: W: 64, ORtg: 123.2 

Query: What are the MOV and Defensive Rating of the Denver Nuggets?
Denver Nuggets: MOV: 5.26, DRtg: 113.0 

Query: Show me the Pythagorean Wins and True Shooting Percentage for Oklahoma City Thunder
Oklahoma City Thunder: W: 57, PW: 58.0, TSpct: 0.608 

Query: How many games have the New York Knicks won and lost?
New York Knicks: W: 50, L: 32 

Query: Give me the Home and Offensive Rating for Boston Celtics and Denver Nuggets
Boston Celtics: ORtg: 123.2, Arena: TD Garden Denver Nuggets: ORtg: 118.5, Arena: Ball Arena 

Query: Give me the Clutch Win Percentage for Boston Celtics and Denver Nuggets
Boston Celtics: WINpct: 63.6 Denver Nuggets: WINpct: 65.0 

Query: Give me the Age of Jimmy Butler
Jimmy Butler: Age: 34 



#### Testing complex queries
Complex queries contain a multitude of stats where the whole database has to be searched. There might also lie a focus of intent on one particular stat, which can be difficult to understand for many language models, in the following we will check the complex queries on our base model of hRAG.

In [22]:
complex_queries = [
    "Give me the Offensive Win Shares and Defensive Win Shares for the player with the most minutes played.",
    "Which player has the highest PER and what are their Win Shares?",
    "Show me the player with the best True Shooting Percentage and their total minutes played.",
    "Find the player with the highest Usage Percentage and list their Points and Assists.",
    "Who has the best Defensive Box Plus-Minus and what is their total Rebounds?",
    "Give me the player with the highest Box Plus-Minus and their VORP."
]

# Process each query and print the result
for complex_query in complex_queries:
    result = process_query(complex_query)
    print(f"Query: {complex_query}")
    print(result, "\n")

Query: Give me the Offensive Win Shares and Defensive Win Shares for the player with the most minutes played.
Teams or players not found in the dataset. 

Query: Which player has the highest PER and what are their Win Shares?
Teams or players not found in the dataset. 

Query: Show me the player with the best True Shooting Percentage and their total minutes played.
Teams or players not found in the dataset. 

Query: Find the player with the highest Usage Percentage and list their Points and Assists.
Teams or players not found in the dataset. 

Query: Who has the best Defensive Box Plus-Minus and what is their total Rebounds?
Teams or players not found in the dataset. 

Query: Give me the player with the highest Box Plus-Minus and their VORP.
Teams or players not found in the dataset. 



## Adaption to complex queries

As seen above, the base version of RAG has Problems looking for a multitude of stats where the questions asked can also have varying meaning and intent. To allow the hRAG to adapt to situations like these, I started using the API of OPENAI to retrieve the needed primary stat, intent and secondary stats for players and teams. I keep Elasticsearch for Data ingestion and retrieval, use GPT to parse the query and understand user intent -> by that extract the relevant entities, this has been tried as a POC with using spacy and bert and has been working partly. But the system got outmatched as soon as I used more complex queries and training (even on hundreds of new questions) has been difficult, since the tabular data has been working mostly on statistics, which was has been increasingly difficult to vectorize and train the model on
For that reason Data Retrieval stays with Elasticsearch but use GPT3.5 turbo to get intent of the prompt and response with human-readable information and allow for data retrieval and humanlike answers.


## RAG System
Using OPEN AI to dissect sentences and improve the accuracy of the question answering - additionally I decided to combine the abbreviation dictionaries to make it easier to handle for the API

In [23]:
os.environ["OPENAI_API_KEY"] = '***'

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [24]:
combined_abbreviation_dict = {**advanced_abbreviation_dict, **clutch_abbreviation_dict, **playoff_player_abbreviation_dict}

#### Parsing and Prompthandling

In [25]:
def parse_query_with_gpt(query):
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"""
        Parse the following query and extract the following information in JSON format:
        - intent: The main action or comparison being requested (e.g., highest, lowest, compare, most, least, predict, matchup).
        - entities: List all the teams or players mentioned in the query, specifying if they are teams or players.
        - primary_metric: The primary metric that determines the main comparison or ranking in the query.
        - stats: List all the statistical metrics mentioned in the query, with the primary metric listed first.
        - clutch: A boolean flag indicating if the query involves clutch stats (true or false).

        Example:
        Query: "Which team had the highest Offensive Rating and how many wins did they achieve?"
        Response: {{
            "intent": "highest",
            "entities": ["team"],
            "primary_metric": "Offensive Rating",
            "stats": ["Offensive Rating", "Wins"],
            "clutch": false
        }}

        Query: "Show me the team which had the lowest Margin of Victory and its True Shooting Percentage"
        Response: {{
            "intent": "lowest",
            "entities": ["team"],
            "primary_metric": "Margin of Victory",
            "stats": ["Margin of Victory", "True Shooting Percentage"],
            "clutch": false
        }}

        Query: "Give me the team with the best Win Percentage and Free Throws Made in clutch situations."
        Response: {{
            "intent": "best",
            "entities": ["team"],
            "primary_metric": "Margin of Victory",
            "stats": ["Margin of Victory", "Defensive Rebound Percentage"],
            "clutch": true
        }},

        Query: "Give me the team Win Percentage of the Dallas Mavericks and Boston Celtics"
        Response: {{
            "intent": "none",
            "entities": ["Dallas Mavericks", "Boston Celtics"],
            "primary_metric": "Win Percentage",
            "stats": ["Win Percentage"],
            "clutch": false
        }},

        Note: If the intent is "predict" or "matchup", ensure that exactly two teams are specified. If there are not exactly two teams, respond with an error message in the format: {{
            "error": "You must specify exactly two teams for a prediction or matchup."

        Query: "Predict the matchup between the Los Angeles Lakers and the Boston Celtics"
        Response: {{
            "intent": "predict",
            "entities": ["Los Angeles Lakers", "Boston Celtics"],
            "primary_metric": "none",
            "stats": [],
            "clutch": false
        }}.

        Only NBA teams should be considered valid entities. If no stat is specified, respond with an error message in the format: {{
            "error": "Please specify at least one statistical metric you want to know more about."
        }}.

        Now, parse the following query: {query}
        """,
        max_tokens=150,
        temperature=0.5
    )

    print(response)
    return response.choices[0].text.strip()

In [26]:
def extract_information_from_response(response_text):
    try:
        match = re.search(r'Response:\s*(\{.*\})', response_text, re.DOTALL)
        if match:
            json_str = match.group(1)
            # print(f"Extracted JSON String: {json_str}")  # Debugging: Print extracted JSON string
            response_json = json.loads(json_str)
        else:
            print("No JSON found in the response text")
            return '', [], '', [], False
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e}")
        return '', [], '', [], False
    intent = response_json.get('intent', '')
    entities = response_json.get('entities', [])
    primary_metric = response_json.get('primary_metric', '')
    stats = response_json.get('stats', [])
    clutch = response_json.get('clutch', False)
    # print(intent, entities, primary_metric, stats, clutch)
    return intent, entities, primary_metric, stats, clutch

#### Abbreviation mapping

In [27]:
def map_stats_to_abbreviations(stats):
    mapped_stats = set()
    for stat in stats:
        stat_lower = stat.lower()
        for key, values in combined_abbreviation_dict.items():
            if any(stat_lower == value.lower() for value in values):
                mapped_stats.add(key)
                break
    return mapped_stats

#### Elasticsearch functions

In [28]:
def search_player_stats(mapped_stats, entities, primary_metric, intent, clutch="False"):
    if 'player' not in entities:
        print("No player entity found in the query.")
        return

    primary_metric_abbr = map_metric_to_abbreviation(primary_metric)
    mapped_stats.add(primary_metric_abbr)

    #print(f"Mapped Stats: {mapped_stats}")  # debug statement

    must_clauses = [{"exists": {"field": stat}} for stat in mapped_stats]
    sort_order = "desc" if intent in ["highest", "best", "top", "most"] else "asc"
    query_body = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "Player"}},
                    {"exists": {"field": primary_metric_abbr}},  # Ensure the primary metric exists
                    {"bool": {"should": must_clauses, "minimum_should_match": 1}}
                ]
            }
        },
        "size": 337,
        "sort": [
            {primary_metric_abbr: {"order": sort_order}}
        ]
    }


    try:
        response = es.search(index=index_name_player, body=query_body)
    except Exception as e:
        print(f"Search failed: {e}")
        return

    hits = response['hits']['hits']
    results = [
        {
            "Player": hit['_source']['Player'],
            **{stat: hit['_source'][stat] for stat in mapped_stats if stat in hit['_source']}
        }
        for hit in hits
    ]

    if primary_metric:
        results = sorted(results, key=lambda x: x.get(primary_metric_abbr, 0), reverse=(sort_order == "desc"))

    return results[0] if results else {}

In [29]:
def search_clutch_stats(mapped_stats, entities, primary_metric, intent):
    if 'team' not in entities:
        print("No team entity found in the query.")
        return

    primary_metric_abbr = map_metric_to_abbreviation(primary_metric)
    mapped_stats.add(primary_metric_abbr)

    print(f"Mapped Stats: {mapped_stats}")  # Debugging: Print mapped stats

    must_clauses = [{"exists": {"field": stat}} for stat in mapped_stats]
    sort_order = "desc" if intent in ["highest", "best", "top", "most"] else "asc"
    query_body = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "TEAM"}},
                    {"exists": {"field": primary_metric_abbr}},  # Ensure the primary metric exists
                    {"bool": {"should": must_clauses, "minimum_should_match": 1}}
                ]
            }
        },
        "size": 32,
        "sort": [
            {primary_metric_abbr: {"order": sort_order}}
        ]
    }

    print(f"Search Query Body: {query_body}")  # Debugging: Print search query body

    try:
        response = es.search(index=index_name_clutch, body=query_body)
    except Exception as e:
        print(f"Search failed: {e}")
        return

    hits = response['hits']['hits']
    results = [
        {
            "Team": hit['_source']['TEAM'],
            **{stat: hit['_source'][stat] for stat in mapped_stats if stat in hit['_source']}
        }
        for hit in hits
    ]

    print(f"Elasticsearch Response: {response}")  # Debugging: Print Elasticsearch response
    print(f"Search Results: {results}")  # Debugging: Print search results

    if primary_metric:
        results = sorted(results, key=lambda x: x.get(primary_metric_abbr, 0), reverse=(sort_order == "desc"))

    return results[0] if results else {}


In [30]:
def search_team_stats(mapped_stats, entities, primary_metric, intent, clutch=False):
    primary_metric_abbr = map_metric_to_abbreviation(primary_metric)
    mapped_stats.add(primary_metric_abbr)

    must_clauses = [{"exists": {"field": stat}} for stat in mapped_stats]

    index = index_name_clutch if clutch else index_name_team
    entity_field = "TEAM" if clutch else "Team"

    if entities:
        entity_clauses = [{"match": {entity_field: entity}} for entity in entities]
    else:
        entity_clauses = []

    query_body = {
        "query": {
            "bool": {
                "must": [
                    {"bool": {"should": must_clauses, "minimum_should_match": 1}},
                    {"bool": {"should": entity_clauses, "minimum_should_match": 1}}
                ]
            }
        },
        "sort": [
            {primary_metric_abbr: {"order": "desc"}}  # Sort by primary metric in descending order
        ],
        "size": 1  # Limit to the top result
    }

    print(f"Search Query Body: {query_body}")

    try:
        response = es.search(index=index, body=query_body)
    except Exception as e:
        print(f"Search failed: {e}")
        return []

    hits = response['hits']['hits']
    if not hits:
        return []

    top_result = {
        entity_field: hits[0]['_source'][entity_field],
        **{stat: hits[0]['_source'][stat] for stat in mapped_stats if stat in hits[0]['_source']}
    }

    return [top_result]

In [31]:
def predict_matchup_stats(entities):
    advanced_stats_fields = ["ORtg", "DRtg", "NRtg", "Off_eFGpct", "Off_TOVpct", "DEF_DRBpct"]
    clutch_stats_fields = ["PTS", "FGpct", "REB", "AST", "TOV"]

    team_stats = {}

    for team_name in entities:
        advanced_stats_query = {
            "query": {
                "match": {
                    "Team": team_name
                }
            },
            "_source": advanced_stats_fields
        }

        clutch_stats_query = {
            "query": {
                "match": {
                    "TEAM": team_name  # Corrected field name for the clutch stats query
                }
            },
            "_source": clutch_stats_fields
        }

        #print(f"Fetching advanced stats for {team_name}...")
        advanced_stats_response = es.search(index="advanced_stats", body=advanced_stats_query)
        #print(f"Advanced stats response for {team_name}: {advanced_stats_response}")

        #print(f"Fetching clutch stats for {team_name}...")
        clutch_stats_response = es.search(index="clutch_stats", body=clutch_stats_query)
        #print(f"Clutch stats response for {team_name}: {clutch_stats_response}")

        team_stats[team_name] = {}

        if advanced_stats_response['hits']['hits']:
            team_stats[team_name].update(advanced_stats_response['hits']['hits'][0]['_source'])
        else:
            print(f"No advanced stats found for {team_name}.")

        if clutch_stats_response['hits']['hits']:
            clutch_stats = clutch_stats_response['hits']['hits'][0]['_source']
            # Add clutch stats to the team stats without overwriting the team name
            for key, value in clutch_stats.items():
                if key != "TEAM":
                    team_stats[team_name][f"clutch_{key}"] = value
        else:
            print(f"No clutch stats found for {team_name}.")

    #print(f"Combined team stats: {team_stats}")
    return team_stats

#### Data Prediction Handling

In [32]:
# these methods are needed to bring the retun of the predict_matchup_stats into a format which the prepare data model can handle
def extract_stats(team_stats, team_name, prefix):
    stats = team_stats[team_name]
    transformed_stats = {}
    for key, value in stats.items():
        if key.startswith('clutch_'):
            new_key = f"{prefix}clutch_{key.split('clutch_')[1]}"
        else:
            new_key = f"{prefix}{key}"
        transformed_stats[new_key] = value
    return transformed_stats

def combine_stats(home_stats, away_stats):
    combined_stats = {**home_stats, **away_stats}
    return combined_stats

In [33]:
def prepare_data_for_model(team_stats):
    teams = list(team_stats.keys())
    home_team = teams[0]
    away_team = teams[1]

    home_stats = team_stats[home_team]
    away_stats = team_stats[away_team]

    feature_names = [
        'home_ORtg', 'home_DRtg', 'home_NRtg', 'home_Off_eFGpct', 'home_Off_TOVpct', 'home_Def_DRBpct', 'home_clutch_PTS', 'home_clutch_FGpct', 'home_clutch_REB', 'home_clutch_AST', 'home_clutch_TOV', 'away_ORtg', 'away_DRtg', 'away_NRtg', 'away_Off_eFGpct', 'away_Off_TOVpct', 'away_Def_DRBpct', 'away_clutch_PTS', 'away_clutch_FGpct', 'away_clutch_REB', 'away_clutch_AST', 'away_clutch_TOV'
    ]

    data = {}
    for feature in feature_names:
        if feature.startswith('home_'):
            stat_name = feature.split('home_')[1]
            data[feature] = [home_stats.get(stat_name, 0)]
        elif feature.startswith('away_'):
            stat_name = feature.split('away_')[1]
            data[feature] = [away_stats.get(stat_name, 0)]

    return home_team, away_team, pd.DataFrame(data)

In [34]:
# Print out the feature names expected by the model
# feature_names = model.feature_names_in_
# print(f"Feature names expected by the model: {feature_names}")

#### Model Loading

In [35]:
import joblib
model_path = 'gradient_boosting_nba_model_corrected_with_pct.pkl'
model = joblib.load(model_path)

# since I often had issues with wrong model types - check the model type
model_type = type(model)
has_predict_method = hasattr(model, 'predict')

print(f"Loaded object type: {model_type}")
if has_predict_method:
    print("The loaded object is a valid model.")
else:
    raise TypeError("The loaded object is not a model. Please provide a valid model.")

Loaded object type: <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
The loaded object is a valid model.


In [36]:
# this method takes the prediction model and the data from the rag pulled data and returns the prediction if the home team wins or loses
def get_prediction(model, data):
    prediction = model.predict(data)
    print(f"Prediction: {prediction}")
    return prediction

#### Backwards Abbreviation Handling

In [37]:
def get_stat_full_name(abbreviation):
    for key, values in combined_abbreviation_dict.items():
        if abbreviation == key:
            return values[0]  # Return the first value as the full name
    return abbreviation

In [38]:
def map_metric_to_abbreviation(metrics):
    if isinstance(metrics, str):
        metrics = [metrics]

    mapped_metrics = []
    for metric in metrics:
        metric_lower = metric.lower()
        abbreviation = metric  # Default to the original if no match is found
        for key, values in combined_abbreviation_dict.items():
            if any(metric_lower == value.lower() for value in values):
                abbreviation = key
                break
        mapped_metrics.append(abbreviation)

    if len(mapped_metrics) == 1:
        return mapped_metrics[0]
    return mapped_metrics

#### Formatting answer using GPT3.5
The first method sums up the responses which

In [39]:
def format_answer(results, primary_metric, stats):
    if not results:
        return "No results found."

    primary_stat_name = get_stat_full_name(primary_metric)

    # Map the primary metric to its abbreviation
    primary_metric_key = map_metric_to_abbreviation(primary_metric)
    print(f"Primary Metric Key: {primary_metric_key}")

    answers = []
    for result in results:
        print(f"Result: {result}")
        print(f"Available Keys in Result: {result.keys()}")

        primary_stat_value = result.get(primary_metric_key, "N/A")
        print(f"Primary Stat Value: {primary_stat_value}")

        other_stats = [stat for stat in stats if stat != primary_metric]
        other_stat_values = {map_metric_to_abbreviation(stat): result.get(map_metric_to_abbreviation(stat), "N/A") for stat in other_stats}
        print(f"Other Stat Values: {other_stat_values}")

        stats_summary = "\n".join([f"{get_stat_full_name(stat)}: {value}" for stat, value in other_stat_values.items()])
        print(f"Stats Summary: {stats_summary}")

        entity_name = result.get('Team', result.get('Player', 'Unknown'))
        entity_type = 'team' if 'Team' in result else 'player'
        print(f"Entity Name: {entity_name}, Entity Type: {entity_type}")

        prompt = f"""
        Given the {entity_type}'s name, primary metric, and the stats summary, generate a fluid and coherent answer.

        {entity_type.capitalize()}: {entity_name}
        Primary Metric: {primary_stat_name}: {primary_stat_value}
        Stats Summary:
        {stats_summary}

        Answer:
        """

        gpt_response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt.strip(),
            max_tokens=100,
            temperature=0.7
        )

        gpt_answer = gpt_response.choices[0].text.strip()

        if "Sorry, there is not enough context" in gpt_answer:
            answers.append(f"{entity_type.capitalize()}: {entity_name}\n{primary_stat_name}: {primary_stat_value}\n{stats_summary}")
        else:
            answers.append(gpt_answer)

    return "\n\n".join(answers)

In [40]:
def format_matchup_answer(prediction, team_stats):
    # turn numpy array prediction to integer
    if isinstance(prediction, list) and len(prediction) == 1:
        prediction = prediction[0]
    elif isinstance(prediction, np.ndarray) and prediction.size == 1:
        prediction = prediction.item()
    elif not isinstance(prediction, int):
        raise TypeError("Prediction must be an integer, a list containing a single integer, or a NumPy array containing a single integer")

    if prediction not in [0, 1]:
        raise ValueError("Prediction must be 0 or 1")

    # I had lots of issues with this statement = gpt forces it's prompts to always start counting at 0, no matter which prompt or list you pass, that's why I had to hardcode the prediction value for the winner (home team 1, and away team 2)
    teams = list(team_stats.keys())
    if prediction == 0:
        winner = teams[1]
    elif prediction == 1:
        winner = teams[0]
    else:
        raise IndexError("Prediction index is out of bounds for team_stats")

    winner_stats = team_stats[winner]

    stats_summary = "\n".join([f"{get_stat_full_name(stat)}: {value}" for stat, value in winner_stats.items() if stat != 'Team'])

    gpt_response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"""
        The prediction indicates that the {winner} is the winning team in the matchup. Based on this prediction, generate a fluid and coherent answer summarizing their performance.

        Winning Team: {winner}
        Stats Summary:
        {stats_summary}

        Answer should start with "The {winner} have won the match. Here are their stats: "
        """,
        max_tokens=300,
        temperature=0.7
    )

    gpt_answer = gpt_response.choices[0].text.strip()

    if "Sorry, there is not enough context" in gpt_answer:
        return f"Winning Team: {winner}\n{stats_summary}"
    else:
        return gpt_answer

In [53]:
# Main execution
complex_queries = [
    "Give me the Offensive Win Shares and Defensive Win Shares for the player with the most minutes played.",
    "Which player has the highest PER and what are their Win Shares?",
    "Show me the player with the best True Shooting Percentage and their total minutes played.",
    "Find the player with the highest Usage Percentage and list their Points and Assists.",
    "Give me the player with the highest Box Plus-Minus and their VORP."
]

for query in complex_queries:
    response_text = parse_query_with_gpt(query)  # Assuming this returns a response_text
    intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
    if intent in ['predict', 'matchup']:
        intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
        home_team, away_team, data = prepare_data_for_model(predict_matchup_stats(entities))
        statistics = predict_matchup_stats(entities)
        prediction = get_prediction(model, data)
        team_answer = predict_matchup_stats(entities)
        print(f"Prediction: {prediction}, Team Answer: {team_answer}")
        format_matchup_answer(prediction, team_answer)
        answer = format_matchup_answer(prediction, team_answer)
        print(f"Answer:\n{answer}\n")
    else:
        mapped_stats = map_stats_to_abbreviations(stats)

        specific_teams = [entity for entity in entities if entity not in ["team", "player"]]

        if 'player' in entities and 'team' in entities:
            team_result = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
            player_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
            team_answer = format_answer(team_result, primary_metric, stats)
            print(team_answer)
            player_answer = format_answer(player_result, primary_metric, stats)
            answer = f"Team Stats:\n{team_answer}\n\nPlayer Stats:\n{player_answer}"
        elif 'player' in entities:
            top_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
            answer = format_answer([top_result], primary_metric, stats)
        elif 'team' in entities or specific_teams:
            top_results = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
            answer = format_answer(top_results, primary_metric, stats)
        else:
            answer = "No valid entity found for search."

        print(f"Query: {query}")
        print(f"Answer:\n{answer}\n")

Completion(id='cmpl-9oGZxZnDf0KRNxdPlGlDohCNbUUeY', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n        Response: {\n            "intent": "most",\n            "entities": ["player"],\n            "primary_metric": "Minutes Played",\n            "stats": ["Offensive Win Shares", "Defensive Win Shares"],\n            "clutch": false\n        }')], created=1721767157, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=608, total_tokens=661))
Primary Metric Key: MP
Result: {'Player': 'Paolo Banchero', 'MP': 2799, 'DWS': 4.0, 'OWS': 1.3}
Available Keys in Result: dict_keys(['Player', 'MP', 'DWS', 'OWS'])
Primary Stat Value: 2799
Other Stat Values: {'OWS': 1.3, 'DWS': 4.0}
Stats Summary: Offensive Win Shares: 1.3
Defensive Win Shares: 4.0
Entity Name: Paolo Banchero, Entity Type: player
Query: Give me the Offensive Win Shares and Defensive Win Shares for the player w

In [44]:
# Main execution
complex_queries = [
    "Predict the matchup between the Dallas Mavericks and the Toronto Raptors",
    "Predict the winner of the matchup between the Boston Celtics and the Dallas Mavericks",
    "Who will win the matchup between the Denver Nuggets and Minnesota Timberwolves"
]

for query in complex_queries:
    response_text = parse_query_with_gpt(query)  # Assuming this returns a response_text
    intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
    if intent in ['predict', 'matchup']:
        intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
        home_team, away_team, data = prepare_data_for_model(predict_matchup_stats(entities))
        statistics = predict_matchup_stats(entities)
        prediction = get_prediction(model, data)
        team_answer = predict_matchup_stats(entities)
        print(f"Prediction: {prediction}, Team Answer: {team_answer}")
        format_matchup_answer(prediction, team_answer)
        answer = format_matchup_answer(prediction, team_answer)
        print(f"Answer:\n{answer}\n")
    else:
        mapped_stats = map_stats_to_abbreviations(stats)

        specific_teams = [entity for entity in entities if entity not in ["team", "player"]]

        if 'player' in entities and 'team' in entities:
            team_result = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
            player_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
            team_answer = format_answer(team_result, primary_metric, stats)
            print(team_answer)
            player_answer = format_answer(player_result, primary_metric, stats)
            answer = f"Team Stats:\n{team_answer}\n\nPlayer Stats:\n{player_answer}"
        elif 'player' in entities:
            top_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
            answer = format_answer([top_result], primary_metric, stats)
        elif 'team' in entities or specific_teams:
            top_results = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
            answer = format_answer(top_results, primary_metric, stats)
        else:
            answer = "No valid entity found for search."

        print(f"Query: {query}")
        print(f"Answer:\n{answer}\n")

Completion(id='cmpl-9oGPZehGha0wiZvrbbdvT9Gi43A9L', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n        Response: {\n            "intent": "predict",\n            "entities": ["Dallas Mavericks", "Toronto Raptors"],\n            "primary_metric": "none",\n            "stats": [],\n            "clutch": false\n        }')], created=1721766513, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=46, prompt_tokens=601, total_tokens=647))
Prediction: [1]
Prediction: [1], Team Answer: {'Dallas Mavericks': {'Off_eFGpct': 0.562, 'DRtg': 115.4, 'NRtg': 2.2, 'ORtg': 117.6, 'Off_TOVpct': 11.2, 'clutch_AST': 1.8, 'clutch_TOV': 0.5, 'clutch_FGpct': 47.5, 'clutch_PTS': 9.4, 'clutch_REB': 3.0}, 'Toronto Raptors': {'Off_eFGpct': 0.535, 'DRtg': 118.8, 'NRtg': -6.5, 'ORtg': 112.3, 'Off_TOVpct': 12.4, 'clutch_AST': 1.6, 'clutch_TOV': 1.0, 'clutch_FGpct': 45.1, 'clutch_PTS': 8.2, 'clutch_REB': 3.7}}

In [42]:
def process_complex_queries(complex_queries):
    for query in complex_queries:
        response_text = parse_query_with_gpt(query)  # Assuming this returns a response_text
        intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
        if intent in ['predict', 'matchup']:
            home_team, away_team, data = prepare_data_for_model(predict_matchup_stats(entities))
            statistics = predict_matchup_stats(entities)
            prediction = get_prediction(model, data)
            team_answer = predict_matchup_stats(entities)
            answer = format_matchup_answer(prediction, team_answer)
            print(f"Prediction: {prediction}, Team Answer: {team_answer}")
            print(f"Answer:\n{answer}\n")
        else:
            mapped_stats = map_stats_to_abbreviations(stats)

            specific_teams = [entity for entity in entities if entity not in ["team", "player"]]

            if 'player' in entities and 'team' in entities:
                team_result = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
                player_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
                team_answer = format_answer(team_result, primary_metric, stats)
                player_answer = format_answer(player_result, primary_metric, stats)
                answer = f"Team Stats:\n{team_answer}\n\nPlayer Stats:\n{player_answer}"
            elif 'player' in entities:
                top_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
                answer = format_answer([top_result], primary_metric, stats)
            elif 'team' in entities or specific_teams:
                top_results = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
                answer = format_answer(top_results, primary_metric, stats)
            else:
                answer = "No valid entity found for search."

            print(f"Query: {query}")
            print(f"Answer:\n{answer}\n")

In [43]:
# Example usage
# "Give me the team with the most losses and their Margin of Victory",
# "Predict the matchup between the Denver Nuggets and Dallas Mavericks"
complex_queries = [
    "Show me the player with the best True Shooting Percentage and their total minutes played."
]

process_complex_queries(complex_queries)

Completion(id='cmpl-9rKu0c4Svbk1EAJIvhFIBudehTNay', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n        Response: {\n            "intent": "best",\n            "entities": ["player"],\n            "primary_metric": "True Shooting Percentage",\n            "stats": ["True Shooting Percentage", "Total Minutes Played"],\n            "clutch": false\n        }')], created=1722498760, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=52, prompt_tokens=605, total_tokens=657))
Primary Metric Key: TSpct
Result: {'Player': 'Drew Peterson', 'MP': 23, 'TSpct': 0.917}
Available Keys in Result: dict_keys(['Player', 'MP', 'TSpct'])
Primary Stat Value: 0.917
Other Stat Values: {'MP': 23}
Stats Summary: Minutes Played: 23
Entity Name: Drew Peterson, Entity Type: player
Query: Show me the player with the best True Shooting Percentage and their total minutes played.
Answer:
Drew Peterson has been

In [66]:
# Example usage
complex_queries = [
    "Give me the team with the most losses and their Margin of Victory"
]

process_complex_queries(complex_queries)

Completion(id='cmpl-9oGkafSku4X67rIVvvKe5utK4MXZ6', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n        Response: {\n            "intent": "most",\n            "entities": ["team"],\n            "primary_metric": "Losses",\n            "stats": ["Losses", "Margin of Victory"],\n            "clutch": false\n        }')], created=1721767816, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=603, total_tokens=653))
Search Query Body: {'query': {'bool': {'must': [{'bool': {'should': [{'exists': {'field': 'MOV'}}, {'exists': {'field': 'L'}}], 'minimum_should_match': 1}}, {'bool': {'should': [], 'minimum_should_match': 1}}]}}, 'sort': [{'L': {'order': 'desc'}}], 'size': 1000}
Primary Metric Key: L
Result: {'Team': 'Detroit Pistons', 'MOV': -9.11, 'L': 68}
Available Keys in Result: dict_keys(['Team', 'MOV', 'L'])
Primary Stat Value: 68
Other Stat Values: {'MOV': -9.11

In [50]:
# Main execution
complex_queries = [
    "Give me the Offensive Rating and Defensive Rating of the Minnesota Timberwolves",
    "Who is the player with the highest Offensive Win Shares and what are their Minutes Played?"
]

for query in complex_queries:
    response_text = parse_query_with_gpt(query)  # Assuming this returns a response_text
    intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
    if intent in ['predict', 'matchup']:
        intent, entities, primary_metric, stats, clutch = extract_information_from_response(response_text)
        print(intent, entities, primary_metric, stats, clutch)
        home_team, away_team, data = prepare_data_for_model(predict_matchup_stats(entities))
        statistics = predict_matchup_stats(entities)
        prediction = get_prediction(model, data)
        team_answer = predict_matchup_stats(entities)
        print(team_answer)
        answer = f"Team Stats:\n{team_answer}\n\nPrediction:\n{prediction}"
    else:
        mapped_stats = map_stats_to_abbreviations(stats)

        specific_teams = [entity for entity in entities if entity not in ["team", "player"]]

        if 'player' in entities and 'team' in entities:
            team_result = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
            player_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
            team_answer = format_answer(team_result, primary_metric, stats)
            print(team_answer)
            player_answer = format_answer(player_result, primary_metric, stats)
            answer = f"Team Stats:\n{team_answer}\n\nPlayer Stats:\n{player_answer}"
        elif 'player' in entities:
            top_result = search_player_stats(mapped_stats, entities, primary_metric, intent)
            answer = format_answer([top_result], primary_metric, stats)
        elif 'team' in entities or specific_teams:
            top_results = search_team_stats(mapped_stats, specific_teams, primary_metric, intent, clutch)
            print(f"Top Results: {top_results}")
            answer = format_answer(top_results, primary_metric, stats)
        else:
            answer = "No valid entity found for search."

        print(f"Query: {query}")
        print(f"Answer:\n{answer}\n")

Completion(id='cmpl-9oGVDEK5Cq2SPqvUyVLyunBu54dwd', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n        Response: {\n            "intent": "none",\n            "entities": ["Minnesota Timberwolves"],\n            "primary_metric": "none",\n            "stats": ["Offensive Rating", "Defensive Rating"],\n            "clutch": false\n        }')], created=1721766863, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=604, total_tokens=657))
Search Query Body: {'query': {'bool': {'must': [{'bool': {'should': [{'exists': {'field': 'DRtg'}}, {'exists': {'field': 'none'}}, {'exists': {'field': 'ORtg'}}], 'minimum_should_match': 1}}, {'bool': {'should': [{'match': {'Team': 'Minnesota Timberwolves'}}], 'minimum_should_match': 1}}]}}, 'size': 1000}
Top Results: [{'Team': 'Minnesota Timberwolves', 'DRtg': 109.0, 'ORtg': 115.6}]
Primary Metric Key: none
Result: {'Team': 'Mi

#### POC - Gradient Boosting Model Usage with RAG
This is the buildup of the prediction engine as seen before to better understand the workings of the function, I decided to keep the building process of it inside the notebook. This feature can be enhanced in the future by including live search functions which scrape the data of injured players during runtime and then lower the value of certain predictions by the amount of a certain stat (for instance: VORP (value over replacement) or PER (player efficiciency rating)) if certain players are missing. The better the player and the better the stats against the matchup - the higher the weight.

In [1099]:
def fetch_team_stats(team_names):
    advanced_stats_fields = ["Team", "ORtg", "DRtg", "NRtg", "Off_eFGpct", "Off_TOVpct", "DEF_DRBpct"]
    clutch_stats_fields = ["PTS", "FGpct", "REB", "AST", "TOV"]

    team_stats = {}

    for team_name in team_names:
        advanced_stats_query = {
            "query": {
                "match": {
                    "Team": team_name
                }
            },
            "_source": advanced_stats_fields
        }

        clutch_stats_query = {
            "query": {
                "match": {
                    "TEAM": team_name  # Corrected field name for the clutch stats query
                }
            },
            "_source": clutch_stats_fields
        }

        advanced_stats_response = es.search(index="advanced_stats", body=advanced_stats_query)
        #print(f"Advanced stats response for {team_name}: {advanced_stats_response}")

        clutch_stats_response = es.search(index="clutch_stats", body=clutch_stats_query)
        #print(f"Clutch stats response for {team_name}: {clutch_stats_response}")

        team_stats[team_name] = {}

        if advanced_stats_response['hits']['hits']:
            #print(f"Advanced stats found for {team_name}.")
            team_stats[team_name].update(advanced_stats_response['hits']['hits'][0]['_source'])
        else:
            print(f"No advanced stats found for {team_name}.")

        if clutch_stats_response['hits']['hits']:
            clutch_stats = clutch_stats_response['hits']['hits'][0]['_source']
            # had to handle clutch stats seperately again with the different Team name
            for key, value in clutch_stats.items():
                if key != "TEAM":
                    team_stats[team_name][f"clutch_{key}"] = value
        else:
            print(f"No clutch stats found for {team_name}.")

    return team_stats

In [889]:
import joblib

# Load the Gradient Boosting model using joblib
model_path = 'gradient_boosting_nba_model_corrected.pkl'
model = joblib.load(model_path)

# Check the type and attributes of the loaded object
model_type = type(model)
has_predict_method = hasattr(model, 'predict')

print(f"Loaded object type: {model_type}")
if has_predict_method:
    print("The loaded object is a valid model.")
else:
    raise TypeError("The loaded object is not a model. Please provide a valid model.")

Loaded object type: <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
The loaded object is a valid model.


In [875]:
# Print out the feature names expected by the model
feature_names = model.feature_names_in_
print(f"Feature names expected by the model: {feature_names}")

Feature names expected by the model: ['home_ORtg' 'home_DRtg' 'home_NRtg' 'home_Off_eFG%' 'home_Off_TOV%'
 'home_Def_DRB%' 'home_clutch_PTS' 'home_clutch_FG%' 'home_clutch_REB'
 'home_clutch_AST' 'home_clutch_TOV' 'away_ORtg' 'away_DRtg' 'away_NRtg'
 'away_Off_eFG%' 'away_Off_TOV%' 'away_Def_DRB%' 'away_clutch_PTS'
 'away_clutch_FG%' 'away_clutch_REB' 'away_clutch_AST' 'away_clutch_TOV']


In [893]:
# Fetch team stats
team_names = ["Los Angeles Lakers", "Chicago Bulls"]
team_stats = fetch_team_stats(team_names)

Fetching advanced stats for Los Angeles Lakers...
Advanced stats response for Los Angeles Lakers: {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 7.121379, 'hits': [{'_index': 'advanced_stats', '_type': '_doc', '_id': '18', '_score': 7.121379, '_source': {'Off_eFGpct': 0.566, 'DRtg': 115.3, 'NRtg': 0.6, 'ORtg': 115.9, 'Off_TOVpct': 12.5, 'Team': 'Los Angeles Lakers'}}, {'_index': 'advanced_stats', '_type': '_doc', '_id': '6', '_score': 4.4468403, '_source': {'Off_eFGpct': 0.561, 'DRtg': 115.4, 'NRtg': 3.4, 'ORtg': 118.8, 'Off_TOVpct': 12.0, 'Team': 'Los Angeles Clippers'}}]}}
Fetching clutch stats for Los Angeles Lakers...
Clutch stats response for Los Angeles Lakers: {'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 7.121379, 'hits': [{'_index': 'clutch_stats

In [891]:
# Prepare the data for the model
def prepare_data_for_model(team_stats, home_team, away_team):
    home_stats = team_stats[home_team]
    away_stats = team_stats[away_team]

    data = {}
    for feature in feature_names:
        if feature.startswith('home_'):
            stat_name = feature.split('home_')[1]
            if stat_name.startswith('clutch_'):
                stat_name = stat_name.split('clutch_')[1]
            data[feature] = [home_stats.get(stat_name, 0)]
        elif feature.startswith('away_'):
            stat_name = feature.split('away_')[1]
            if stat_name.startswith('clutch_'):
                stat_name = stat_name.split('clutch_')[1]
            data[feature] = [away_stats.get(stat_name, 0)]

    return pd.DataFrame(data)

In [894]:
# Prepare data for the teams
home_team = "Los Angeles Lakers"
away_team = "Chicago Bulls"
data = prepare_data_for_model(team_stats, home_team, away_team)

# Predict using the model
prediction = model.predict(data)
print(f"Prediction: {prediction}")

# Compare the predictions
comparison = {
    home_team: prediction[0],
    away_team: prediction[1] if len(prediction) > 1 else "N/A"
}

print(f"Comparison: {comparison}")

Prediction: [1]
Comparison: {'Los Angeles Lakers': 1, 'Chicago Bulls': 'N/A'}


#### Helper Method to check indices

In [ ]:
complex_queries = [
    "Give me the Offensive Win Shares and Defensive Win Shares for the player with the most minutes played.",
    "Which player has the highest PER and what are their Win Shares?",
    "Show me the player with the best True Shooting Percentage and their total minutes played.",
    "Find the player with the highest Usage Percentage and list their Points and Assists.",
    "Who has the best Defensive Box Plus-Minus and what is their total Rebounds?",
    "Give me the player with the highest Box Plus-Minus and their VORP."
]

complex_queries = [
    "Give me the team with the best Margin of Victory and Defensive Rebound Percentage.",
    "Which Team has the Highest Net Rating and Offensive Rating?",
    "Show me the team with the top True Shooting Percentage and Effective Field Goal Percentage for Offense.",
    "Show me the Team which had the lowest Margin of Victory and its True Shooting Percentage"
]

#### Helper Methods

In [845]:
# checking mappings for the chosen index
player_index_mappings = es.indices.get_mapping(index='clutch_stats')
print("Index Mappings:")
print(player_index_mappings)
# clutch_stats      player_stats        advanced_stats
sample_doc = es.search(index='clutch_stats', size=1)
print("Sample Document from Index:")
print(sample_doc['hits']['hits'][0]['_source'])

Index Mappings:
{'clutch_stats': {'mappings': {'properties': {'+/-': {'type': 'float'}, '3PA': {'type': 'float'}, '3PM': {'type': 'float'}, '3Ppct': {'type': 'float'}, 'AST': {'type': 'float'}, 'BLK': {'type': 'float'}, 'BLKA': {'type': 'float'}, 'DREB': {'type': 'float'}, 'FGA': {'type': 'float'}, 'FGM': {'type': 'float'}, 'FGpct': {'type': 'float'}, 'FTA': {'type': 'float'}, 'FTM': {'type': 'float'}, 'FTpct': {'type': 'float'}, 'GP': {'type': 'long'}, 'L': {'type': 'long'}, 'MIN': {'type': 'float'}, 'OREB': {'type': 'float'}, 'PF': {'type': 'float'}, 'PFD': {'type': 'float'}, 'PTS': {'type': 'float'}, 'REB': {'type': 'float'}, 'STL': {'type': 'float'}, 'TEAM': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'TOV': {'type': 'float'}, 'W': {'type': 'long'}, 'WINpct': {'type': 'float'}, 'embedding': {'type': 'float'}, 'text': {'properties': {'+/-': {'type': 'float'}, '3PA': {'type': 'float'}, '3PM': {'type': 'float'}, '3Ppct': {'type': 'float'}, 'AST':

In [458]:
def count_entries_in_player_stats_index():
    query_body = {
        "query": {
            "match_all": {}
        }
    }
    response = es.count(index="player_stats", body=query_body)

    print(f"Total number of entries in player_stats index: {response['count']}")
    return response['count']
total_entries = count_entries_in_player_stats_index()
print(f"Total entries in player_stats: {total_entries}")

Total number of entries in player_stats index: 337
Total entries in player_stats: 337


In [167]:
# add in conference functionality -> redundant
def get_teams_by_conference(conference, conference_data):
    print(f"Fetching teams for conference: {conference}")
    teams = []
    for division, teams_list in conference_data[conference].items():
        teams.extend(teams_list)
    return teams

In [702]:
def print_index_mappings(index_name):
    try:
        mappings = es.indices.get_mapping(index=index_name)
        print(f"Mappings for index {index_name}: {mappings}")
    except Exception as e:
        print(f"Failed to get mappings for index {index_name}: {e}")

print_index_mappings(index_name_team)

Mappings for index advanced_stats: {'advanced_stats': {'mappings': {'properties': {'3PAr': {'type': 'float'}, 'Age': {'type': 'float'}, 'Arena': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'Attend': {'type': 'long'}, 'Attend/G': {'type': 'long'}, 'DRtg': {'type': 'float'}, 'Def_DRBpct': {'type': 'float'}, 'Def_FT/FGA': {'type': 'float'}, 'Def_TOVpct': {'type': 'float'}, 'Def_eFGpct': {'type': 'float'}, 'FTr': {'type': 'float'}, 'L': {'type': 'long'}, 'MOV': {'type': 'float'}, 'NRtg': {'type': 'float'}, 'ORtg': {'type': 'float'}, 'Off_FT/FGA': {'type': 'float'}, 'Off_ORBpct': {'type': 'float'}, 'Off_TOVpct': {'type': 'float'}, 'Off_eFGpct': {'type': 'float'}, 'PL': {'type': 'long'}, 'PW': {'type': 'long'}, 'Pace': {'type': 'float'}, 'SOS': {'type': 'float'}, 'SRS': {'type': 'float'}, 'TSpct': {'type': 'float'}, 'Team': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'W': {'type': 'long'}, 'embedding': {'type': 'fl

In [833]:
import joblib

In [834]:
# Load pre-trained machine learning model
model = joblib.load('data/gradient_boosting_model.pkl')

def retrieve_documents(query, index='your-index'):
    response = es.search(index=index, body={
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title", "content"]
            }
        }
    })
    return [hit['_source'] for hit in response['hits']['hits']]

def generate_gpt3_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']

def make_prediction(features):
    # Assuming the model expects a list of features
    prediction = model.predict([features])
    return prediction[0]

def main(query, features):
    # Step 1: Retrieve documents from Elasticsearch
    documents = retrieve_documents(query)
    retrieved_content = ' '.join([doc['content'] for doc in documents])

    # Step 2: Generate prompt for GPT-3.5 Turbo
    prompt = f"Using the following documents:\n{retrieved_content}\nAnd given the features {features}, what can you tell me about the expected outcome?"

    # Step 3: Make prediction using the ML model
    prediction = make_prediction(features)
    prompt += f"\nThe prediction from the model is: {prediction}"

    # Step 4: Generate response from GPT-3.5 Turbo
    response = generate_gpt3_response(prompt)
    print(response)

In [838]:
# Load the model and feature importances
gb_model = joblib.load('data/gradient_boosting_model.pkl')
feature_importances = joblib.load('data/feature_importances.pkl')

# Save the model to a file
model_file_path = 'saved_models/gradient_boosting_model.pkl'
feature_importances_path = 'saved_models/feature_importances.json'

joblib.dump(gb_model, model_file_path)

# Save feature importances to a JSON file
with open(feature_importances_path, 'w') as f:
    json.dump(feature_importances.tolist(), f)

In [839]:
# Index the model file paths in Elasticsearch
model_doc = {
    'model_file_path': model_file_path,
    'feature_importances_path': feature_importances_path
}

# Index the model document
es.index(index='models', id='gradient_boosting_model', body=model_doc)

print("Model file paths indexed successfully.")

Model file paths indexed successfully.
